In [ ]:
from pyspark.sql import SparkSession

# Initialiser une session Spark
spark = SparkSession.builder.appName("ImageSelection").getOrCreate()

# Définir le répertoire contenant les images
image_dir = 'images'

# Créer un RDD à partir de la liste des noms d'images
image_names_rdd = spark.sparkContext.parallelize(selected_images)

# Filtrer les noms d'images existants
existing_images_rdd = image_names_rdd.filter(lambda image_name: os.path.exists(os.path.join(image_dir, image_name)))

# Mapper pour ouvrir les images avec la bibliothèque PIL
def open_image(image_name):
    image_path = os.path.join(image_dir, image_name)
    image = Image.open(image_path)
    return (image_name, image)

opened_images_rdd = existing_images_rdd.map(open_image)

# Reducer pour demander les balises aux utilisateurs
def add_tags(image):
    tags = input('Entrez des balises pour l\'image {} : '.format(image[0]))
    return (image[0], image[1], tags)

images_with_tags_rdd = opened_images_rdd.map(add_tags)

# Collecter les résultats en tant que liste
selected_images_with_tags = images_with_tags_rdd.collect()

# Fermer la session Spark
spark.stop()

# Afficher les images sélectionnées avec les balises
for image in selected_images_with_tags:
    print("Image :", image[0])
    print("Tags :", image[2])
    print()
